In [1]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', 1000)

In [3]:
## Load Data

In [4]:
data_dir = '..\data\\'
cvadr = pd.read_parquet(data_dir + 'final_dataset.parquet')
cvadr.head(2)

,DRUG_NAME,MASS,ATC_LEVEL_2,ATC_VECTOR,TARGETS,TARGET_VECTOR,REPORT_NO,DATE_RECEIVED,REPORT_TYPE,GENDER,AGE,SERIOUS_FLAG,REPORTER_TYPE,SOURCE,WEIGHT_KG,ADMIN_ROUTE,DOSAGE_FORM,CONCOMITANT_DRUGS,REACTION,SOC,SIDER
0,BIVALIRUDIN,2180.2853,{'ANTITHROMBOTIC AGENTS': 1},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",{'Prothrombin': 1},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",226246,2007-12-07,Spontaneous,Male,76.0,No,Other Health Professional,MAH,74.0,Intravenous (not otherwise specified),POWDER FOR SOLUTION INTRAVENOUS,1.0,Drug ineffective,General disorders and administration site cond...,False
1,BIVALIRUDIN,2180.2853,{'ANTITHROMBOTIC AGENTS': 1},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",{'Prothrombin': 1},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",436056,2012-05-09,Spontaneous,Female,53.0,Yes,Physician,MAH,NaN,Intravenous (not otherwise specified),POWDER FOR SOLUTION INTRAVENOUS,6.0,Bradycardia,Cardiac disorders,False


In [6]:
# name first row as primary key

cvadr = cvadr.reset_index()
cvadr = cvadr.rename(columns={'index': 'ROW_ID'})
cvadr.head(2)

# change DATE_RECEIVED to datetime tpye
cvadr['DATE_RECEIVED'] = pd.to_datetime(cvadr['DATE_RECEIVED'] , yearfirst=True)

# fix dates in the future (these were converted to the wrong century)
cvadr['DATE_RECEIVED'] = cvadr['DATE_RECEIVED'].apply(lambda x: x if x.year < 2020 else x - pd.DateOffset(years=100))

# change ADMIN_ROUTE and DOSAGE_FORM to category type

cvadr[['ADMIN_ROUTE', 'DOSAGE_FORM']] = cvadr[['ADMIN_ROUTE', 'DOSAGE_FORM']].astype('category')

In [ ]:
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

NameError: name 'y_test' is not defined